In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

In [ ]:
sns.set(color_codes=True)

In [ ]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
other_df = pd.get_dummies(train['DetectedCamera']) 
other_df_test = pd.get_dummies(test['DetectedCamera'])

In [ ]:
merged_df = pd.concat([other_df,train],1) 
merged_df_test = pd.concat([other_df_test,test],1)

In [ ]:
le = LabelEncoder()
merged_df['SignFacing (Target)']=le.fit_transform(merged_df['SignFacing (Target)'])


In [ ]:
train = merged_df 
test = merged_df_test

In [ ]:
train_label = train['SignFacing (Target)']

training = train.drop(labels=['Id', 'DetectedCamera', 'SignFacing (Target)'], axis=1)
testing = test.drop(labels=['Id', 'DetectedCamera'], axis=1)

In [ ]:
#It wont work. Dont know why
training['sin'] = np.sin(training['AngleOfSign'])
training['cos'] = np.cos(training['AngleOfSign'])

testing['sin'] = np.sin(testing['AngleOfSign'])
testing['cos'] = np.cos(testing['AngleOfSign'])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import xgboost as xgb


## Grid Search

In [ ]:
# kfold = KFold(n_splits=3, random_state=7)
# model = xgb.XGBClassifier(objective='multi:softprob', nthread=-1, reg_lambda=1, seed=2017, gamma=0.2, learning_rate=0.15, max_depth=4, min_child_weight=4, colsample_bylevel=0.6, colsample_bytree=1)
# params_grid = { 'n_estimators':list([60,70,75,80,85]), 'reg_lambda':[5,6,7,8,9,10], 'subsample':[0.6,0.7,0.8,0.9,0.99]  }
# grid1 = GridSearchCV(estimator=model, param_grid=params_grid, n_jobs=-1, scoring='log_loss', cv=kfold)
# grid1.fit(training, train_label)

In [ ]:
# print grid1.best_estimator_
# print grid1.best_params_
# print grid1.best_score_
# print grid1.best_index_

In [ ]:
# new_model = grid1.best_estimator_
# new_model.fit(training, train_label)
# prediction = new_model.predict_proba(testing)

## Prep For XGB

In [ ]:
xtrain, xval, ytrain, yval = train_test_split(training,train_label,test_size=0.33,random_state=7)

In [ ]:
preds = testing.columns
DMtraining = xgb.DMatrix(training, train_label, feature_names=preds)
DMtrain = xgb.DMatrix(xtrain,ytrain,feature_names=preds)
DMval = xgb.DMatrix(xval,yval,feature_names=preds)
DMtest = xgb.DMatrix(testing, feature_names=preds)

In [ ]:
xgb_params={
    'eta':0.15,
    'max_depth':4,
    'colsample_bytree':1.0,
    'colsample_bylevel':0.6,
    'subsample' : 0.9,
    'objective':'multi:softprob',
    'eval_metric':'mlogloss',
    'min_child_weight':1,
    'silent':1,
    'seed':2017,
    'nthread':4,
    'gamma' : 0.2,
    'num_class':4,
    'lambda':7,
    'alpha':0
}

In [ ]:
watchlist = [(DMtrain, 'Train'), (DMval,'Validation')]
num_rounds= 2500
model = xgb.train(xgb_params, DMtrain, num_rounds, watchlist, early_stopping_rounds=80, verbose_eval=5)

In [ ]:
model = xgb.train(xgb_params, DMtraining,num_boost_round=int(82/95))
prediction = model.predict(DMtest)

In [ ]:
subm_file = pd.read_csv('sample_submission.csv')

In [ ]:
subm_file['Id'] = test['Id']
subm_file['Front'] = prediction[:,0]
subm_file['Left'] = prediction[:,1]
subm_file['Rear'] = prediction[:,2]
subm_file['Right'] = prediction[:,3]

In [ ]:
subm_file.to_csv('subm_new3.csv', index=None)